In [1]:
!pip install zemberek-python
#NOTE:
#print(dir(class))=this get function about it

In [2]:
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

In [3]:
morphology = TurkishMorphology.create_with_defaults()

2025-01-07 14:03:05,169 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 3.9031198024749756



In [4]:
# SPELLING SUGGESTION or spelling correction
spell_checker = TurkishSpellChecker(morphology)

In [5]:
# Function to correct spelling in each row's tweet
def correct_tweet(tweet, index,df):
    try:
        # Ensure the tweet is a string
        if not isinstance(tweet, str):
            return tweet
        # Split the tweet into words
        words = tweet.split()
        corrected_words = []
        for word in words:
            word = word.strip()  # Remove leading/trailing spaces
            corrected_word = spell_checker.suggest_for_word(word)  # Get suggestions for the word
            if corrected_word:
                #print(f"Original: {word} -> Corrected: {corrected_word[0]}")  # Log input and output
                corrected_words.append(corrected_word[0])  # Use the first suggestion
                
            else:
                #print(f"Original: {word} -> Corrected: {word} (No suggestion)")
                corrected_words.append(word)  # Keep the original word if no suggestions
        # Join the corrected words back into a sentence
        corrected_tweet = ' '.join(corrected_words)
        # Update the DataFrame directly
        df.loc[index, 'corrected_tweet'] = corrected_tweet
        #print("After Turkish Spell Checker",corrected_tweet)
        return corrected_tweet
    except Exception as e:
        print(f"Error processing tweet: {tweet}, Error: {e}")
        return tweet  # Return the original tweet if an error occurs

## Frist Spelling suggestion or correction and then normaliziton

In [7]:
import pandas as pd
df = pd.read_csv('Llama_tokenized.csv')
df = df[["etiket", "tweet"]]

In [8]:
pd.set_option("display.max_colwidth", None) 

In [10]:
if df.isnull().values.any():
    print(df.isnull().sum())
    df.dropna(inplace=True)
print(df.isnull().sum())

etiket     0
tweet     20
dtype: int64
etiket    0
tweet     0
dtype: int64


In [12]:
df.sample(10, random_state=42)

,etiket,tweet
6079,nefret,suriyelilerin profillerini ülkeden istemiyorum tweet atmış bir şerefsizi
5150,hiçbiri,türk süvarileri gerçekten büyük iddiaları vardı sadece 5 eylül 1 günde 3 önemli şehri aldıkları değil halkın direnişi atatürk in liderliği sayesinde yunanlıları alt edip gerçek bir türk suvarisi aziz türk milletinin kurtuluşu büyük mücadeleler verdikleri anı saymak lazım
4912,nefret,artık yahudi ermeni rum kanlıları yönetiyormuş bir havada geziyorsunuz
4034,nefret,bunları yapan gavur tohumudur yunanin arkada bıraktığı pislikleridir
132,nefret,sevmek türk anneleri sikiyor gerçekleşmicek yazık
4605,saldırgan,hainolanı işbirlikçisini tanımayız
1246,nefret,türklerin birliği türk ırkının onuru elimizden geleni yapacağız
264,saldırgan,almanyayı baştan başa tanrı bizimle afişleriyle donatan hitlere bile ateist diyor salak musevi allah bizimle hıristiyan allah bizimle müslüman allah bizimle hindu tanrılar bizimle din ideoloji ikisi aynı bok
4046,nefret,kadar psikolojik olarak çarpık olan birini anlamak bile imkânsız gibiydi kendi kendine faydalı olamayan düşük değerde bir bencildim
353,saldırgan,süper alçaklar


## Spell Correction

In [14]:
# Add a new column for corrected tweets
df['corrected_tweet'] = None  # Initialize with original tweets

# Process each row individually
for idx, row in df.iterrows():
    #print(row['tweet'])
    correct_tweet(row['tweet'], idx,df)

In [15]:
df.sample(10, random_state=42)

,etiket,tweet,corrected_tweet
6079,nefret,suriyelilerin profillerini ülkeden istemiyorum tweet atmış bir şerefsizi,Suriye'lileri profillerinin ülkede istemiyoruz tweet almış bin şerefsiz
5150,hiçbiri,türk süvarileri gerçekten büyük iddiaları vardı sadece 5 eylül 1 günde 3 önemli şehri aldıkları değil halkın direnişi atatürk in liderliği sayesinde yunanlıları alt edip gerçek bir türk suvarisi aziz türk milletinin kurtuluşu büyük mücadeleler verdikleri anı saymak lazım,türk süvarileri gerçekten büyük iddiaları vardı sadece o eylül o günde o önemli şehir aldıkları değil halkın direniş atatürk en liderliği sayesinde yunanlılar ait edip gerek bin türk Suvari'ni aziz türk milletinin kuruluşu büyük mücadeleler verdikleri aynı satmak lazım
4912,nefret,artık yahudi ermeni rum kanlıları yönetiyormuş bir havada geziyorsunuz,artık yahudi ermeni rus zanlıları önetiyormuş bin davada geliyorsunuz
4034,nefret,bunları yapan gavur tohumudur yunanin arkada bıraktığı pislikleridir,bunlar yapan gavur tohumundur Yune'nin arada bıraktığı Pais'likleridir
132,nefret,sevmek türk anneleri sikiyor gerçekleşmicek yazık,seçmek türk anneler siliyor gerçekleşmicek yazık
4605,saldırgan,hainolanı işbirlikçisini tanımayız,hainolanı işbirliksisini tanımayı
1246,nefret,türklerin birliği türk ırkının onuru elimizden geleni yapacağız,türlerin birliği türk ırkını onur elimizden gelen yapacağız
264,saldırgan,almanyayı baştan başa tanrı bizimle afişleriyle donatan hitlere bile ateist diyor salak musevi allah bizimle hıristiyan allah bizimle müslüman allah bizimle hindu tanrılar bizimle din ideoloji ikisi aynı bok,Almanya'ya başkan başka tanı bizimle afişeleriyle donatan hitler ile ateist ediyor salak musavi allah bizimle hristiyan allah bizimle müslüman allah bizimle hindi tanrıyar bizimle bin ideolojik ikili aynı çok
4046,nefret,kadar psikolojik olarak çarpık olan birini anlamak bile imkânsız gibiydi kendi kendine faydalı olamayan düşük değerde bir bencildim,kadar psikolojik olarak çarpık olan birinci anlamak ile imkansız gibiydi kendi kendini faydalı olmayan düşük değere bin bencildin
353,saldırgan,süper alçaklar,süper alçaklar


In [16]:
# SENTENCE NORMALIZATION
normalizer = TurkishSentenceNormalizer(morphology)

In [18]:
if df.isnull().values.any():
    print(df.isnull().sum())
    df.dropna(inplace=True)
print(df.isnull().sum())

etiket             0
tweet              0
corrected_tweet    0
dtype: int64


## Normalize 

In [20]:
df['correct_normalize_tweet'] = None 
try:
    for index, row in df.iterrows():
        normalize = normalizer.normalize(row['corrected_tweet'])
        df.at[index, 'correct_normalize_tweet'] = normalize
except Exception as e:
        print(f"Satır {index} işlenirken hata oluştu: {e}")

In [21]:
df.sample(10, random_state=42)

,etiket,tweet,corrected_tweet,correct_normalize_tweet
6079,nefret,suriyelilerin profillerini ülkeden istemiyorum tweet atmış bir şerefsizi,Suriye'lileri profillerinin ülkede istemiyoruz tweet almış bin şerefsiz,suriyelileri profillerinin ülkede istemiyoruz tweet almış bin şerefsiz
5150,hiçbiri,türk süvarileri gerçekten büyük iddiaları vardı sadece 5 eylül 1 günde 3 önemli şehri aldıkları değil halkın direnişi atatürk in liderliği sayesinde yunanlıları alt edip gerçek bir türk suvarisi aziz türk milletinin kurtuluşu büyük mücadeleler verdikleri anı saymak lazım,türk süvarileri gerçekten büyük iddiaları vardı sadece o eylül o günde o önemli şehir aldıkları değil halkın direniş atatürk en liderliği sayesinde yunanlılar ait edip gerek bin türk Suvari'ni aziz türk milletinin kuruluşu büyük mücadeleler verdikleri aynı satmak lazım,türk süvarileri gerçekten büyük iddiaları vardı sadece o eylül o günde o önemli şehir aldıkları değil halkın direniş atatürk en liderliği sayesinde yunanlılar ait edip gerek bin türk süvarini aziz türk milletinin kuruluşu büyük mücadeleler verdikleri aynı satmak lazım
4912,nefret,artık yahudi ermeni rum kanlıları yönetiyormuş bir havada geziyorsunuz,artık yahudi ermeni rus zanlıları önetiyormuş bin davada geliyorsunuz,artık yahudi ermeni rus zanlıları önetiyormuş bin davada geliyorsunuz
4034,nefret,bunları yapan gavur tohumudur yunanin arkada bıraktığı pislikleridir,bunlar yapan gavur tohumundur Yune'nin arada bıraktığı Pais'likleridir,bunlar yapan gavur tohumundur yunenin arada bıraktığı paislikleridir
132,nefret,sevmek türk anneleri sikiyor gerçekleşmicek yazık,seçmek türk anneler siliyor gerçekleşmicek yazık,seçmek türk anneler siliyor gerçekleşmiçek yazık
4605,saldırgan,hainolanı işbirlikçisini tanımayız,hainolanı işbirliksisini tanımayı,hainolanı işbirliksisini tanımayı
1246,nefret,türklerin birliği türk ırkının onuru elimizden geleni yapacağız,türlerin birliği türk ırkını onur elimizden gelen yapacağız,türlerin birliği türk ırkını onur elimizden gelen yapacağız
264,saldırgan,almanyayı baştan başa tanrı bizimle afişleriyle donatan hitlere bile ateist diyor salak musevi allah bizimle hıristiyan allah bizimle müslüman allah bizimle hindu tanrılar bizimle din ideoloji ikisi aynı bok,Almanya'ya başkan başka tanı bizimle afişeleriyle donatan hitler ile ateist ediyor salak musavi allah bizimle hristiyan allah bizimle müslüman allah bizimle hindi tanrıyar bizimle bin ideolojik ikili aynı çok,almanya'ya başkan başka tanı bizimle afişeleriyle donatan hitler ile ateist ediyor salak müsavi allah bizimle hristiyan allah bizimle müslüman allah bizimle hindi tanrıyar bizimle bin ideolojik ikili aynı çok
4046,nefret,kadar psikolojik olarak çarpık olan birini anlamak bile imkânsız gibiydi kendi kendine faydalı olamayan düşük değerde bir bencildim,kadar psikolojik olarak çarpık olan birinci anlamak ile imkansız gibiydi kendi kendini faydalı olmayan düşük değere bin bencildin,kadar psikolojik olarak çarpık olan birinci anlamak ile imkansız gibiydi kendi kendini faydalı olmayan düşük değere bin bencildin
353,saldırgan,süper alçaklar,süper alçaklar,süper alacaklar


In [22]:
df.to_csv('Llama_HateSpeechTurkish.csv', index=False, encoding='utf-8-sig')

In [36]:
data = pd.read_csv("HateSpeechTurkish.csv")

In [38]:
data = pd.concat([df,data], axis=0)

In [44]:
data.drop(columns='row_id', inplace=True)

In [46]:
data

,etiket,tweet,corrected_tweet,correct_normalize_tweet
0,nefret,avrupa kafasıyla imzalanmış bir sözleşmeye göre kökenli toplummuşuz abi sözleşmesi müslümanları hristiyanları ayırmasın kadını koruyor illa şeyi dine bağlayacaksınız amk,avrupa kazasıyla imzalamış bin sözleşmeye göre kökenli toplunmuşuz abd sözleşmesi müslümanlar hristiyanlar ayırması kadın koyuyor ila şey diye sağlayacaksınız ama,avrupa kazasıyla imzalamış bin sözleşmeye göre kökenli toplunmuşuz abd sözleşmesi müslümanlar hristiyanlar ayırması kadın koyuyor ila şey diye sağlayacaksınız ama
1,nefret,avrupa nın kafasızca imzaladığı bir sözleşmeyle bizi kökenli olarak tanımlayanlar var sözleşmesi müslümanları hristiyanlardan ayırma saçmalık değil mi abi,avrupa ın kafasızda imzaladığı bin sözleşmeye biz kökenli olarak tanımayanlar var sözleşmesi müslümanlar Hristiyan'larda ayırmak saçmalık değil ki abd,avrupa in kafasızda imzaladığı bin sözleşmeye biz kökenli olarak tanımayanlar var sözleşmesi müslümanlar hristiyanlarda ayırmak saçmalık değil ki abd
2,nefret,kadını koruma dine sığınmak lazım insan yapalım amk,kadın koruma diye sığınak lazım insan yapalım ama,kadın koruma diye sığınak lazım insan yapalım ama
3,nefret,ülkede oturup türk vatandaşı olup türkler 800900yüzyılları müslümanlığa geçmiş olmasına rağmen hala müslümanlıkla sorunu olan varsa bşka ülkeye gitsin hristiyan yahudi şaman budist ataist olmak istiyosa olsunyeteter karışmayın müslümanlara salın aq,ülkede oturup türk vatandaş olur türker 800900yüzyılları müslümanlığı geçiş olmasını rağmen hava müslümanlıkla sorun olan varsa başka ülkede gitsin hristiyan yahudi zaman budist ateist olmak istiyorsa olsunyeteter karışmayın müslümanlar sayın ay,ülkede oturup türk vatandaş olur türker 800900yüzyılları müslümanlığı geçiş olmasını rağmen hava müslümanlıkla sorun olan varsa başka ülkede gitsin hristiyan yahudi zaman budist ateist olmak istiyorsa olsunyeteter karışmayın müslümanlar sayın ay
4,nefret,müslümanlarla problemi olan türklerin başka ülkeye gitmesi gerekir,müslümanlara problem olan türlerin başka ülkede girmesi gerekir,müslümanlara problem olan türlerin başka ülkede girmesi gerekir
...,...,...,...,...
10219,hiçbiri,müslüman bir ülkeden hristiyan bir ülkeye gidiyorsan oranın yaşam tarzına ver kurallarına uymak zorundasın yasağı koyanları tebrik ederim,müslüman bin ülkede hristiyan bin ülkede gidiyorsun oranı yaşam tarzını var kurallarına duymak zorundasın yasağı koyunları tebrik ederim,müslüman bin ülkede hristiyan bin ülkede gidiyorsun oranı yaşam tarzını var kurallarına duymak zorundasın yasağı koyunları tebrik ederim
10220,hiçbiri,kadının dini dili ırkı olur gün bari faşistlik yapmayın gün bari ırkçılık yapmayın türk kürt alevi sünni müslüman hristiyan yahudi bütün kadınların günü kutlu olsun be kardeşim,kadının din dile ırkçı olur gün biri faşistlik yapmayı gün biri ırkçılık yapmayı türk kürt alevi sunni müslüman hristiyan yahudi bütün kadınların gün mutlu olsun ve kardeşi,kadının din dile ırkçı olur gün biri faşistlik yapmayı gün biri ırkçılık yapmayı türk kürt alevi sünni müslüman hristiyan yahudi bütün kadınların gün mutlu olsun ve kardeşi
10221,hiçbiri,mor renk eskiden istridyeden yapılabildiği pahalıymış yüzden krallar zengin soylular hristiyan din adamları giyermiş papa önemli günlerde yine mor giyer taç törenlerinde giyilen renk iktidarın gücün sembolüdür,zor denk eskiden istiridyeden yapılabildiği pahasıymış yüzde kurallar zengin soydular hristiyan bin adamları gidermiş para önemli günlerde yine zor gider tam törenlerine girilen denk iktidarı gücü sembolüdür,zor denk eskiden istiridyeden yapılabildiği pahasıymış yüzde kurallar zengin soydular hristiyan bin adamları gidermiş para önemli günlerde yine zor gider tam törenlerine girilen denk iktidarı gücü sembolüdür
10222,hiçbiri,taktığı haç a takıladım neticede hristiyan olabilir hepsinde olsaydı evet garip derdim özel kuvvet konseptinin içinin böyle boşaltılmasına takıldım çevik kuvveti bile yol güvenliği böyle dizdiğinizde haliyle biraz yadırgarla

In [48]:
data.to_csv('TrLlama_HateSpeech.csv', index=False, encoding='utf-8-sig')

# Note for Tokenize

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# İlk çalıştırmada aşağıdaki iki satırı uncomment ederek çalıştırın:
# nltk.download('punkt')
# nltk.download('stopwords')

def tokinize_and_filter(sentence):
    # Cümleyi tokene çevir
    tokens = word_tokenize(sentence, language='turkish')
    
    # Türkçe stopwords'leri al
    turkish_stopwords = set(stopwords.words('turkish'))
    
    # Stopwords ve noktalama işaretlerini çıkar
    filtered_tokens = [word for word in tokens if word.lower() not in turkish_stopwords and word.isalnum()]
    
    return filtered_tokens

# Tokinize edilmiş kelimeleri stringe dönüştür
def tokens_to_string(tokens):
    return ' '.join(tokens)

# Örnek cümle
sentence = "İnsan isek, her kesimin derdi ile haklı oldukları uğurda destek olmalıyız! Haksızlığa karşı susan dilsiz şeytandır. Kürt halkı Türk halkının düşmanı değildir. Asla olmadı. PKK ile Kürt halkını bir tutmayın! O insanlar da yıllardır zorluk çekiyor ve şiddet görüyor."

# Tokinize edilmiş ve filtrelenmiş kelimeler
filtered_words = tokinize_and_filter(sentence)

# String olarak birleştirilmiş çıktı
filtered_string = tokens_to_string(filtered_words)

print("Sonuç String:", filtered_string)


# Note for Zemberek

In [ ]:
# Create a new column to store the results
df['stems'] = None

# Process each row in the DataFrame
for index, row in df.iterrows():
    sentence = row['tweet']
    
    try:
        # Analyze the sentence and disambiguate
        analysis = morphology.analyze_sentence(sentence)
        after = morphology.disambiguate(sentence, analysis)
        
        # Extract stem information from the disambiguated results
        stems = [s.get_stem() for s in after.best_analysis() if len(s.get_stem()) > 1]
        
        # Save the stem results in the new column
        df.at[index, 'stems'] = ' '.join(stems)
    
    except Exception as e:
        # Handle errors and save the error message in the 'stems' column
        print(f"Error processing row {index}: {e}")
        #df.at[index, 'stems'] = str(e)

In [ ]:
# SPELLING SUGGESTION
sc = TurkishSpellChecker(morphology)

In [ ]:

li = ["okuyablirim", "tartısıyor", "Ankarada", "knlıca", "yapablrim", "kıredi", "geldm", "geliyom", "aldm", "asln"]
for word in li:
    print(word + " --> " + ' '.join(sc.suggest_for_word(word)))

In [ ]:
print(normalizer.normalize(" suriye li lerle"))

In [ ]:
# SENTENCE BOUNDARY DETECTION
extractor = TurkishSentenceExtractor()
print("Extractor instance created in: ")

text = "İnsanoğlu aslında ne para ne sevgi ne kariyer ne şöhret ne de çevre ile sonsuza dek mutlu olabilecek bir " \
       "yapıya sahiptir. Dış kaynaklardan gelebilecek bu mutluluklar sadece belirli bir zaman için insanı mutlu " \
       "kılıyor. Kişi bu kaynakları elde ettiği zaman belirli bir dönem için kendini iyi hissediyor, ancak alışma " \
       "dönemine girdiği andan itibaren bu iyilik hali hızla tükeniyor. Mutlu olma sanatının özü bu değildir. Gerçek " \
       "mutluluk, kişinin her türlü olaya ve duruma karşı kendini pozitif tutarak mutlu hissedebilmesi halidir. Bu " \
       "davranış şeklini edinen insan, zor günlerde güçlü, mutlu günlerde zevk alan biri olur ve mutluluğu kalıcı " \
       "kılar. "

sentences = extractor.from_paragraph(text)
print(f"Sentences separated")

for sentence in sentences:
    print(sentence)
print("\n")

In [ ]:
# TOKENIZATION
tokenizer = TurkishTokenizer.DEFAULT

tokens = tokenizer.tokenize("Saat'e 12:00.")
for token in tokens:
    print('Content = ', token.content)
    print('Type = ', token.type_.name)
    print('Start = ', token.start)
    print('Stop = ', token.end, '\n')

In [ ]:
# SINGLE WORD MORPHOLOGICAL ANALYSIS
results = morphology.analyze("şeytandır")
for result in results:
    print(result)
print("\n")

In [ ]:
#print(dir(class))=this get function about it

In [ ]:
# SENTENCE ANALYSIS AND DISAMBIGUATION

sentence = "İnsan olmak hakkındadır herşey."
analysis = morphology.analyze_sentence(sentence)
after = morphology.disambiguate(sentence, analysis)

print("\nBefore disambiguation")
for e in analysis:
    print(f"Word = {e.inp}")
    for s in e:
        print(s.format_string())
        print(s.get_stem())

print("\nAfter disambiguation")
for s in after.best_analysis():
    print(s.format_string())
    #print(dir(s))
    print(s.get_stem())
   # print(s.get_morphemes())
    print(s.get_ending())